In [1]:
import requests
from lxml import etree
import re
import math
import pandas as pd
from datetime import datetime

cookies = {
    'car16888_set_provinceId': '31',
    'car16888_set_cityId': '383',
    'car16888_set_area': '359',
    'car16888_set_areaName': '%E6%9D%AD%E5%B7%9E',
    'car16888_set_areaDir': 'hz',
    'car16888_set_iscity': '',
}

headers = {
    'authority': 'xl.16888.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    # 'cookie': 'car16888_set_provinceId=31; car16888_set_cityId=383; car16888_set_area=359; car16888_set_areaName=%E6%9D%AD%E5%B7%9E; car16888_set_areaDir=hz; car16888_set_iscity=',
    'referer': 'https://xl.16888.com/ev-202001-202402-1.html',
    'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
}

In [2]:
#  生成年-月序列的函数
def datelist(beginDate, endDate):
    # beginDate, endDate是形如‘20160601’的字符串或datetime格式
    return [datetime.strftime(x,'%Y%m') for x in list(pd.date_range(start=beginDate, end=endDate, freq="M"))]

In [3]:
# 对应时间区域数据
def everyMonthDate(beginDate, endDate):
    response = requests.get(
        f'https://xl.16888.com/ev-{beginDate}-{endDate}-1.html', cookies=cookies, headers=headers)
    html_text = etree.HTML(response.text)
    # 总页数
    pageTotal = math.ceil(
        int(html_text.xpath('//span[@class="lineBlock va-m"]')[0].text[1:4]) / 50)

    modellist = []
    company = []
    priceRange = []
    salesVolume = []

    for i in range(1, pageTotal + 1):
        response1 = requests.get(
            f'https://xl.16888.com/ev-{beginDate}-{endDate}-{i}.html', cookies=cookies, headers=headers)

        html_text1 = etree.HTML(response1.text)
        trList = html_text1.xpath(
            '//table[@class="xl-table-def xl-table-a"]')[0].xpath('//tr')

        for trs in trList:
            if (trs.getchildren()[1].getchildren()):
                modellist.append(trs.getchildren()[1].getchildren()[0].text)
                company.append(trs.getchildren()[3].getchildren()[0].text)
                priceRange.append(trs.getchildren()[4].getchildren()[0].text)
                salesVolume.append(trs.getchildren()[2].text)
    return pd.DataFrame({"厂商": company, "车型": modellist, "售价（万元）": priceRange, "销量": salesVolume})

In [4]:
# 遍历月份，导出数据表
def getSalesData(beginDate, endDate):
    dfInfos = pd.DataFrame()
    for time in datelist(beginDate, endDate):
        dfInfo = everyMonthDate(time, time)
        dfInfo['年份'] = time[:4]
        dfInfo['月份'] = time[4:6]
        # 将 年-月放到前面
        dfInfo.insert(loc=0, column="年份", value=dfInfo.pop("年份"))
        dfInfo.insert(loc=1, column="月份", value=dfInfo.pop("月份"))
        dfInfos = pd.concat([dfInfos, dfInfo], axis=0)
    dfInfos.to_excel(f'{beginDate}-{endDate}之间的电动车销量.xlsx', index=False)

In [5]:
# 执行
if __name__ == '__main__':
    getSalesData('20200101', '20240229')
    pass

In [1]:
import requests
from lxml import etree
import re
import math
import pandas as pd
from datetime import datetime

cookies = {
    'car16888_set_provinceId': '31',
    'car16888_set_cityId': '383',
    'car16888_set_area': '359',
    'car16888_set_areaName': '%E6%9D%AD%E5%B7%9E',
    'car16888_set_areaDir': 'hz',
    'car16888_set_iscity': '',
}

headers = {
    'authority': 'xl.16888.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    # 'cookie': 'car16888_set_provinceId=31; car16888_set_cityId=383; car16888_set_area=359; car16888_set_areaName=%E6%9D%AD%E5%B7%9E; car16888_set_areaDir=hz; car16888_set_iscity=',
    'referer': 'https://xl.16888.com/style-202001-202402-1.html',
    'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
}

In [2]:
#  生成年-月序列的函数
def datelist(beginDate, endDate):
    # beginDate, endDate是形如‘20160601’的字符串或datetime格式
    return [datetime.strftime(x,'%Y%m') for x in list(pd.date_range(start=beginDate, end=endDate, freq="M"))]

In [3]:
# 对应时间区域数据
def everyMonthDate(beginDate, endDate):
    response = requests.get(
        f'https://xl.16888.com/style-{beginDate}-{endDate}-1.html', cookies=cookies, headers=headers)
    response.encoding = 'utf-8' 
    html_text = etree.HTML(response.text)
    # 总页数
    pageTotal = math.ceil(
        int(html_text.xpath('//span[@class="lineBlock va-m"]')[0].text[1:4]) / 50)

    modellist = []
    company = []
    priceRange = []
    salesVolume = []

    for i in range(1, pageTotal + 1):
        response1 = requests.get(
            f'https://xl.16888.com/style-{beginDate}-{endDate}-{i}.html', cookies=cookies, headers=headers)
        print(response1.text)
        
        html_text1 = etree.HTML(response1.text)
        trList = html_text1.xpath(
            '//table[@class="xl-table-def xl-table-a"]')[0].xpath('//tr')

        for trs in trList:
            if (trs.getchildren()[1].getchildren()):
                modellist.append(trs.getchildren()[1].getchildren()[0].text)
                company.append(trs.getchildren()[3].getchildren()[0].text)
                priceRange.append(trs.getchildren()[4].getchildren()[0].text)
                salesVolume.append(trs.getchildren()[2].text)
    return pd.DataFrame({"厂商": company, "车型": modellist, "售价（万元）": priceRange, "销量": salesVolume})

In [4]:
# 遍历月份，导出数据表
def getSalesData(beginDate, endDate):
    dfInfos = pd.DataFrame()
    for time in datelist(beginDate, endDate):
        dfInfo = everyMonthDate(time, time)
        dfInfo['年份'] = time[:4]
        dfInfo['月份'] = time[4:6]
        # 将 年-月放到前面
        dfInfo.insert(loc=0, column="年份", value=dfInfo.pop("年份"))
        dfInfo.insert(loc=1, column="月份", value=dfInfo.pop("月份"))
        dfInfos = pd.concat([dfInfos, dfInfo], axis=0)
    dfInfos.to_excel(f'{beginDate}-{endDate}之间的车型销量.xlsx', index=False)

In [ ]:
# 执行
if __name__ == '__main__':
    getSalesData('20200101', '20240229')
    pass

<!DOCTYPE HTML>
<html>

<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8" />
<meta name="keywords" content="" />
<meta name="description" content="" />
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
<meta name="renderer" content="webkit">
<title>2020年1月车型销量排行榜，车型销量查询- 车主之家</title>
<link rel="shortcut icon" type="image/x-icon" href="//a.img16888.com/common/images/favicon.ico" />
<link rel="stylesheet" type="text/css" href="//s.img16888.com/dist/css/??common/global.css?t=2022006291007" />
<link rel="stylesheet" type="text/css" href="//s.img16888.com/dist/css/??xl/global.css,xl/index.css?t=2022006291007" />
<!--[if IE 6]>
<link rel="stylesheet" type="text/css" href="//s.img16888.com/dist/css/??xl/ie.css?t=2022006291007" />
<![endif]-->
</head>


<body>
<div class="site-nav">
    <div class="wrapper clearfix">
        <div class="change-city substation-city g-ml200 citylist clearfix f_l">
            <i class="ico"></i>
            <span class="J_su